In [31]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('.'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

./submission_xgb.csv
./titanic.py
./test.csv
./submission.csv
./submission_rf.csv
./train.csv
./gender_submission.csv
./titanic.ipynb
./submission_rfc.csv
./submission_lg.csv
./submission_lr.csv


In [32]:
import warnings
warnings.filterwarnings('ignore')

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error
import xgboost as xgb
import pandas as pd

# Data preprocess: train, test and 
data_path = './'

In [33]:
df_train_orginal_data = pd.read_csv(data_path + 'train.csv')
df_train_removal = df_train_orginal_data.drop(['PassengerId','Name','Ticket','Cabin'], axis=1)
df_train_dummy = pd.get_dummies(df_train_removal)
df_train_dummy.corr()

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
Survived,1.000000,-0.338481,-0.077221,-0.035322,0.081629,0.257307,0.543351,-0.543351,0.168240,0.003650,-0.155660
Pclass,-0.338481,1.000000,-0.369226,0.083081,0.018443,-0.549500,-0.131900,0.131900,-0.243292,0.221009,0.081720
Age,-0.077221,-0.369226,1.000000,-0.308247,-0.189119,0.096067,-0.093254,0.093254,0.036261,-0.022405,-0.032523
SibSp,-0.035322,0.083081,-0.308247,1.000000,0.414838,0.159651,0.114631,-0.114631,-0.059528,-0.026354,0.070941
Parch,0.081629,0.018443,-0.189119,0.414838,1.000000,0.216225,0.245489,-0.245489,-0.011069,-0.081228,0.063036
Fare,0.257307,-0.549500,0.096067,0.159651,0.216225,1.000000,0.182333,-0.182333,0.269335,-0.117216,-0.166603
Sex_female,0.543351,-0.131900,-0.093254,0.114631,0.245489,0.182333,1.000000,-1.000000,0.082853,0.074115,-0.125722
Sex_male,-0.543351,0.131900,0.093254,-0.114631,-0.245489,-0.182333,-1.000000,1.000000,-0.082853,-0.074115,0.125722
Embarked_C,0.168240,-0.243292,0.036261,-0.059528,-0.011069,0.269335,0.082853,-0.082853,1.000000,-0.148258,-0.778359
Embarked_Q,0.003650,0.221009,-0.022405,-0.026354,-0.081228,-0.117216,0.074115,-0.074115,-0.148258,1.000000,-0.496624


In [34]:
X = df_train_dummy.drop(['Survived'], axis=1).fillna(0)
y = df_train_dummy['Survived']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)

In [35]:
df_submission = pd.read_csv(data_path + 'test.csv').drop(['PassengerId','Name','Ticket','Cabin'], axis=1)
df_submission_dummy = pd.get_dummies(df_submission).fillna(0)
df_submission_dummy

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,3,34.5,0,0,7.8292,0,1,0,1,0
1,3,47.0,1,0,7.0000,1,0,0,0,1
2,2,62.0,0,0,9.6875,0,1,0,1,0
3,3,27.0,0,0,8.6625,0,1,0,0,1
4,3,22.0,1,1,12.2875,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...
413,3,0.0,0,0,8.0500,0,1,0,0,1
414,1,39.0,0,0,108.9000,1,0,1,0,0
415,3,38.5,0,0,7.2500,0,1,0,0,1
416,3,0.0,0,0,8.0500,0,1,0,0,1


In [36]:
ID = pd.read_csv(data_path + 'test.csv')['PassengerId']

0       892
1       893
2       894
3       895
4       896
       ... 
413    1305
414    1306
415    1307
416    1308
417    1309
Name: PassengerId, Length: 418, dtype: int64

In [37]:
# Logistic regression
lg = LogisticRegression()
lg.fit(X_train, y_train)
y_pred = lg.predict(X_test)

print('The accuracy_score of logistic regression is {}'.format(accuracy_score(y_test, y_pred)))

submission_pred = lg.predict(df_submission_dummy)

submission = pd.concat([pd.DataFrame(ID), pd.DataFrame(submission_pred)], axis=1)
submission.columns = ['PassengerId','Survived']
submission.to_csv('submission_lg.csv', index=False)

The accuracy_score of logistic regression is 0.7668161434977578


In [38]:
# XGBoost
xgb_classifer = xgb.XGBClassifier()

xgb_classifer.fit(X_train, y_train)
y_pred = xgb_classifer.predict(X_test)
print('The accuracy_score of XGBoost is {}'.format(accuracy_score(y_test, y_pred)))

submissioin_pred = xgb_classifer.predict(df_submission_dummy)

submission = pd.concat([pd.DataFrame(ID), pd.DataFrame(submissioin_pred)], axis=1)
submission.columns = ['PassengerId','Survived']
submission.to_csv('submission_xgb.csv', index=False)

[21:35:17] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
The accuracy_score of XGBoost is 0.7937219730941704


In [39]:
# RandomForest
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(max_depth=10)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
print('RandomForest score: {}'.format(accuracy_score(y_test, y_pred)))

submissioin_pred = rf.predict(df_submission_dummy)

submission = pd.concat([pd.DataFrame(ID), pd.DataFrame(submissioin_pred)], axis=1)
submission.columns = ['PassengerId','Survived']
submission.to_csv('submission_rf.csv', index=False)

RandomForest score: 0.820627802690583
